In [62]:
# import base libraries
import numpy as np
import pandas as pd
import geopandas as gpd

# import dashboard libraries
import panel as pn
pn.extension('tabulator')
import geoviews.tile_sources as gvts
import holoviews as hv
import hvplot.pandas
from holoviews import opts
from holoviews.plotting.links import DataLink

In [63]:
# adjust pandas settings to view all columns
pd.set_option("display.max_columns", None)

In [64]:
# read in starting data
# change everything after'data/' to the name of the new spreadsheet
responses = pd.read_csv("data/Map My CBO - OC (Responses) - Form Responses 02-22-2023.csv", encoding= 'unicode_escape')

# city boundaries 
cities = gpd.read_file("data/City_Boundaries.geojson", driver='GeoJSON')

In [65]:
# clean spreadsheet data file
# rename columns 
responses = responses.rename(columns={
    'In which Orange County cities does your organization conduct community outreach? Check all that apply.': 'Cities',
    'In which language(s) does your organization conduct outreach? Check all that apply.': 'Outreach languages',
    'Which of the following race/ethnic group(s) describes the communities that your organization reaches? Check all that apply.': 'Race/ethnic groups reached',
    'Which of the following demographic characteristic(s) describe the communities that your organization reaches? Check all that apply.': 'Population demographic characteristics',
    'Which of the following outreach activities does your organization conduct to reach communities? Check all that apply.': 'Outreach activities',
    'If your organization is listed on OC Nonprofit Central, please post the direct link to the profile page here.':'OC Nonprofit Central profile'
})

# select only the columns we want to display in the downloadable table
responses_clean = responses[['Organization Name',
                            'Cities',
                            'Race/ethnic groups reached',
                            'Outreach languages',
                            'Population demographic characteristics',
                            'Outreach activities',
                            'OC Nonprofit Central profile']]

# preview cleaned data
responses_clean.head() # .head() will display the first 5 rows by default. enter a number (i.e. responses_clean.head(20) to preview more)

,Organization Name,Cities,Race/ethnic groups reached,Outreach languages,Population demographic characteristics,Outreach activities,OC Nonprofit Central profile
0,Fullerton Collaborative,Fullerton,"Asian American, Black & African American, Lati...","English, Korean, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","In-person events / gatherings, Mailings, Tabli...",https://www.ocnonprofitcentral.org/organizatio...
1,LGBTQ Center OC,"Aliso Viejo, Anaheim, Brea, Buena Park, Costa ...","Asian American, Black & African American, Lati...","English, Spanish","LGBTQ+, Immigrants / Refugees, Limited English...","In-person events / gatherings, Tabling / flier...",https://www.ocnonprofitcentral.org/organizatio...
2,"Abrazar, Inc.","Anaheim, Brea, Buena Park, Costa Mesa, Cypress...","Asian American, Black & African American, Lati...","English, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","Door-to-door canvassing, In-person events / ga...",NaN
3,The Cambodian Family Community Center,"Anaheim, Costa Mesa, Fountain Valley, Garden G...","Asian American, Latino / Latinx","English, Khmer, Spanish, Vietnamese, Cham","Immigrants / Refugees, Limited English speakin...","Caravan / parade, Door-to-door canvassing, In-...",NaN
4,PHFE WIC Program,"Anaheim, Cypress, La Habra, Placentia, Santa Ana","Asian American, Black & African American, Lati...","English, Arabic, Armenian, Chinese (Cantonese)...","Children under 5, Houseless individuals & fami...","In-person events / gatherings, Virtual events ...",NaN


In [66]:
# clean city boundary data file
cities = cities.drop(['JURISDICTI','Acres','Area_SqMi','OCSurveyDBOCityBoundariesArea'],axis=1) # drop columns we don't need
cities = cities.loc[cities['CITY']!='Unincorporated'] # drop unincorporated territories
cities = cities.drop_duplicates(subset=['CITY'], keep='first') # drop smallest polygons for repeated cities (i.e. tustin)

# set projection
cities = cities.to_crs(4326)

In [67]:
# make join columns match
cities = cities.rename(columns={'CITY':'City'})

# make join column values match
cities['City'] = cities['City'].str.title()

In [68]:
# preview cleaned data
cities.sort_values(['City']) # checking for duplicates

,OBJECTID,City,geometry
42,521,Aliso Viejo,"POLYGON ((-117.71643 33.59765, -117.71682 33.6..."
3,481,Anaheim,"POLYGON ((-117.68346 33.87068, -117.68393 33.8..."
7,485,Brea,"POLYGON ((-117.91191 33.94608, -117.91183 33.9..."
24,503,Buena Park,"POLYGON ((-117.98535 33.89552, -117.98968 33.8..."
19,498,Costa Mesa,"POLYGON ((-117.92944 33.70201, -117.92953 33.7..."
29,508,Cypress,"POLYGON ((-118.05877 33.84620, -118.05906 33.8..."
76,555,Dana Point,"POLYGON ((-117.68678 33.48710, -117.68687 33.4..."
78,557,Fountain Valley,"POLYGON ((-117.91535 33.72883, -117.91520 33.7..."
2,480,Fullerton,"POLYGON ((-117.92435 33.92440, -117.92435 33.9..."
84,563,Garden Grove,"POLYGON ((-117.97194 33.80569, -117.97340 33.8..."


In [69]:
#check number of cities
len(cities)

35

In [70]:
# separate city column in responses data
responses_separated = responses_clean['Cities'].str.split(', ', expand=True)
responses_separated.columns = ['City'+str(i) for i in responses_separated.columns]

# join back to dataframe
responses_separated_concat = pd.concat([responses_clean, responses_separated], axis=1)

# pivot
responses_byCity = pd.melt(responses_separated_concat,
                           id_vars=['Organization Name',
                            'Race/ethnic groups reached',
                            'Outreach languages',
                            'Population demographic characteristics',
                            'Outreach activities',
                            'OC Nonprofit Central profile'],
                           value_vars=responses_separated.columns,
                          var_name='City Number',
                          value_name='City')
# drop separation column
responses_byCity = responses_byCity.drop(['City Number'], axis=1)

# reorder columns
responses_byCity = responses_byCity[['Organization Name',
                            'City',
                            'Race/ethnic groups reached',
                            'Outreach languages',
                            'Population demographic characteristics',
                            'Outreach activities',
                            'OC Nonprofit Central profile']]

# fill NaNs with "none"
responses_byCity = responses_byCity.fillna('None')
responses_byCity.sort_values(['Organization Name']).head(10)

,Organization Name,City,Race/ethnic groups reached,Outreach languages,Population demographic characteristics,Outreach activities,OC Nonprofit Central profile
622,"Abrazar, Inc.",La Habra,"Asian American, Black & African American, Lati...","English, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","Door-to-door canvassing, In-person events / ga...",None
808,"Abrazar, Inc.",Newport Beach,"Asian American, Black & African American, Lati...","English, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","Door-to-door canvassing, In-person events / ga...",None
1490,"Abrazar, Inc.",None,"Asian American, Black & African American, Lati...","English, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","Door-to-door canvassing, In-person events / ga...",None
932,"Abrazar, Inc.",Placentia,"Asian American, Black & African American, Lati...","English, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","Door-to-door canvassing, In-person events / ga...",None
1552,"Abrazar, Inc.",None,"Asian American, Black & African American, Lati...","English, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","Door-to-door canvassing, In-person events / ga...",None
1924,"Abrazar, Inc.",None,"Asian American, Black & African American, Lati...","English, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","Door-to-door canvassing, In-person events / ga...",None
560,"Abrazar, Inc.",Irvine,"Asian American, Black & African American, Lati...","English, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","Door-to-door canvassing, In-person events / ga...",None
1614,"Abrazar, Inc.",None,"Asian American, Black & African American, Lati...","English, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","Door-to-door canvassing, In-person events / ga...",None
684,"Abrazar, Inc.",La Palma,"Asian American, Black & African American, Lati...","English, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","Door-to-door canvassing, In-person events / ga...",None
312,"Abrazar, Inc.",Fountain Valley,"Asian American, Black & African American, Lati...","English, Spanish, Vietnamese","Children under 5, Houseless individuals & fami...","Door-to-door canvassing, In-person events / ga...",None


In [71]:
# drop where city == None
responses_byCity = responses_byCity.loc[responses_byCity['City']!='None']

In [72]:
# join spreadsheet data to city boundaries
responses_joined = cities.merge(responses_byCity, on='City', how='left')

# project
responses_joined = gpd.GeoDataFrame(responses_joined, geometry='geometry')
responses_joined = responses_joined.to_crs(epsg=4326)

In [73]:
# returns the number of rows in the df -- check point
len(responses_joined)

1092

In [74]:
# create a df for the map
# select desired columns
CBOcount = responses_joined[['Organization Name', 'City']]

# group by city and count number of orgs
CBOcount = CBOcount.groupby(['City']).count().reset_index()
CBOcount = CBOcount.drop_duplicates()

# rename count column
CBOcount = CBOcount.rename(columns={'Organization Name': 'Organizations'})

# add back geom
CBOcount_geo = cities.merge(CBOcount, on='City', how='left')
CBOcount_geo.sort_values('City') # check for duplicates

,OBJECTID,City,geometry,Organizations
23,521,Aliso Viejo,"POLYGON ((-117.71643 33.59765, -117.71682 33.6...",30
3,481,Anaheim,"POLYGON ((-117.68346 33.87068, -117.68393 33.8...",49
7,485,Brea,"POLYGON ((-117.91191 33.94608, -117.91183 33.9...",31
15,503,Buena Park,"POLYGON ((-117.98535 33.89552, -117.98968 33.8...",37
11,498,Costa Mesa,"POLYGON ((-117.92944 33.70201, -117.92953 33.7...",34
18,508,Cypress,"POLYGON ((-118.05877 33.84620, -118.05906 33.8...",29
28,555,Dana Point,"POLYGON ((-117.68678 33.48710, -117.68687 33.4...",22
30,557,Fountain Valley,"POLYGON ((-117.91535 33.72883, -117.91520 33.7...",31
2,480,Fullerton,"POLYGON ((-117.92435 33.92440, -117.92435 33.9...",43
34,563,Garden Grove,"POLYGON ((-117.97194 33.80569, -117.97340 33.8...",45


In [75]:
len(CBOcount_geo)

35

In [76]:
#make map
map00 = CBOcount_geo.hvplot(
c='Organizations',
frame_width=500,
frame_height=400,
dynamic=True,
geo=True,
#crs=6426,-- parsing errors with this projection for some reason now. Converting to 4326 prior to creating map to fix.
hover_cols=['City'],
cmap='Blues',
line_color='gray',
title='Number of Organizations by City')

basemap = gvts.CartoLight.opts(alpha=0.6)

map01=basemap*map00.opts(xaxis=None, yaxis=None)

In [77]:
# create Organization Details table
filter_table=pn.widgets.Tabulator(responses_byCity, 
                                  layout='fit_columns',
                                  pagination='remote',
                                  page_size=11,
                                  width=1200)

In [78]:
#create file name box and download button
filename, button = filter_table.download_menu(
    text_kwargs={'name':'Enter filename', 'value':'outreach-oc.csv'},
    button_kwargs={'name':'Download table'}
)

In [79]:
#creating type widgets
city_filter = pn.widgets.TextInput(name='Search by city name', value='', width=200)
HTC_filter = pn.widgets.TextInput(name='Search by population demographic characteristics', value='', width=300)
RE_filter = pn.widgets.TextInput(name='Search by race/ethnic groups reached', value='', width=250)
lang_filter = pn.widgets.TextInput(name='Search by outreach languages', value='', width=200)
act_filter = pn.widgets.TextInput(name='Search by outreach activities', value='', width=200)

def contains_filter(df, pattern, column):
    if not pattern:
        return df
    return df[df[column].str.contains(pattern, case=False, na=False, regex=True)]
    
filter_table.add_filter(pn.bind(contains_filter, pattern=city_filter, column='City'))
filter_table.add_filter(pn.bind(contains_filter, pattern=RE_filter, column='Race/ethnic groups reached'))
filter_table.add_filter(pn.bind(contains_filter, pattern=lang_filter, column='Outreach languages'))
filter_table.add_filter(pn.bind(contains_filter, pattern=HTC_filter, column='Population demographic characteristic'))
filter_table.add_filter(pn.bind(contains_filter, pattern=act_filter, column='Outreach Activities'))

In [80]:
with open('TITLE_ABOUT.md','r') as file:
    title = file.read()
    print(title)
    file.close()

# Map My CBO<br>
<span style="font-size:16px;">
Map My CBO is a free public search tool designed to support partnership, collaboration, and community outreach for local organizations that work with the diverse communities of Orange County. Easy to update and free to use, the map and directory provides regional stakeholders with a unique, point-in-time view of the regional nonprofit and grassroots network, including:

<li><span style="font-size:16px;">Services provided, where and in what languages; and</li>
<li><span style="font-size:16px;">Community outreach and civic engagement capabilities</li>

<span style="font-size:16px;">This map and table directory of community-based organizations builds on outreach data collected during the 2020 Census and is searchable by city, population demographic characteristics, and outreach activities.<br>
<br>
The more organizations that are reflected in this map, the more powerful the tool can be to drive partnerships for community outreach, engagement

In [81]:
with open('RE.md','r') as file:
    REtext = file.read()
    print(REtext)
    file.close()

<br>
<br>
<b><span style="font-size:16px;">Searchable Categories</b><br>
<b>Race/ethnic groups reached</b>
<li> Asian American</li>
<li> Black & African American</li>
<li> Latino / Latinx</li>
<li> Middle Eastern & North African</li>
<li> Native American & Tribal Communities &emsp; &emsp; &emsp;</li>
<li> Native Hawaiian & Pacific Islander</li>
<li> Non-Hispanic White</li>
<br>
<br>



In [82]:
with open('HTC.md','r') as file:
    HTCtext = file.read()
    print(HTCtext)
    file.close()

<br>
<br>
<br>
<b> Population demographic characteristics &emsp; &emsp; &emsp;</b>
<li> Children under 5</li>
<li> Farmworkers</li>
<li> Houseless individuals & families</li>
<li> Immigrants / refugees</li>
<li> LGBTQ+</li>
<li> Limited English speaking ability</li>
<li> Low broadband subscription rate</li>
<li> Older adults / seniors</li>
<li> People with disabilities</li>
<li> Veterans</li>
<br>
<br>



In [83]:
with open('ACT.md','r') as file:
    ACTtext = file.read()
    print(ACTtext)
    file.close()

<br>
<br>
<br>
<b> Outreach activities </b>
<li> Caravan / parade</li>
<li> Door-to-door canvassing</li>
<li> In-person events / gatherings</li>
<li> Food / kit distribution</li>
<li> Mailings</li>
<li> Media (TV, radio, newspapers)</li>
<li> Peer-to-peer / mass texting</li>
<li> Phone banking</li>
<li> Tabling / flier distribution</li>
<li> Virtual events / gatherings</li>
<br>
<br>



In [84]:
with open('LANG.md','r') as file:
    LANGtext = file.read()
    print(LANGtext)
    file.close()

<br>
<br>
<br>
<b>Outreach languages</b>
<li> English</li>
<li> Arabic</li>
<li> Armenian</li>
<li> Cham</li>
<li> Chinese (Cantonese)  &emsp; &emsp; &emsp;</li>
<li> Chinese (Mandarin)</li>
<li> Farsi</li>
<li> Hindi</li>
<li> Japanese</li>
<li> Khmer</li>
<li> Korean</li>
<li> Russian</li>
<li> Spanish</li>
<li> Tagalog</li>
<li> Vietnamese</li>
<li> Other</li>
<br>
<br>



In [85]:
with open('MAP.md','r') as file:
    mapText = file.read()
    print(mapText)
    file.close()

<br>
<br>
<span style="font-size:16px;">Click a city on the map to see how many organizations in this directory support community outreach in that city.<br>
<br>
 The darker the color of the city on the map, the more organizations that support community outreach.<br>
<br>
<br>


In [86]:
with open('ORG_DETAILS.md','r') as file:
    orgDetailsText = file.read()
    print(orgDetailsText)
    file.close()

<br>
<b><span style="font-size:16px;">Organization Details</b><br>
<ul><span style="font-size:16px;">
<li>Enter keywords into the relevant search fields and press "enter" to filter the information.</li>
<li>Enter one category into a search field at a time (i.e., one city, one language, etc.). The full list of searchable categories is at the bottom of this page.</li>
<li>Delete any words in the search fields and press "enter" to reset the filter.</li>
<li>Click the column heading (e.g., "City" or "Organization Name") to sort the list in alphabetical order.</li>
</span>
<br>



In [87]:
with open('DOWNLOAD.md','r') as file:
    dlText = file.read()
    print(dlText)
    file.close()

<br><b><span style="font-size:16px;">Download Table:</b>
<span style="font-size:16px;">Enter a filename relevant to your search (e.g. Spanish canvassing in Anaheim), and then click "Download table" to save a spreadsheet file onto your device.
</span>



In [88]:
with open('BOTTOM.md','r') as file:
    bottomText = file.read()
    print(bottomText)
    file.close()

<br>
<br>
<br>
Hosted by [Charitable Ventures of Orange County](https://charitableventuresoc.org/)<br>
Contact: cbo-directory@charitableventuresoc.org<br>
Developed by [Good Work Collaborative](https://www.ourgoodwork.co/) (2022)<br>
Last update: 02-22-2023<br>



In [89]:
pn.extension()

dash3 = pn.Column(title,
                pn.Row(map01, mapText),
                orgDetailsText,
                pn.Row(city_filter,
                         RE_filter,
                         lang_filter,
                         HTC_filter,
                         act_filter),
                filter_table,
                dlText,
                filename,
                button,
                pn.Row(REtext, LANGtext, HTCtext, ACTtext),
                bottomText).servable()

In [90]:
dash3

Column
    [0] Markdown(str)
    [1] Row
        [0] HoloViews(Overlay)
        [1] Markdown(str)
    [2] Markdown(str)
    [3] Row
        [0] TextInput(name='Search by city name', width=200)
        [1] TextInput(name='Search by race/ethnic g..., width=250)
        [2] TextInput(name='Search by outreach l..., width=200)
        [3] TextInput(name='Search by population d..., width=300)
        [4] TextInput(name='Search by outreach a..., width=200)
    [4] Tabulator(layout='fit_columns', page_size=11, pagination='remote', value=                          ..., width=1200)
    [5] Markdown(str)
    [6] TextInput(name='Filename', value='table.csv')
    [7] Button(name='Download table')
    [8] Row
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Markdown(str)
    [9] Markdown(str)